In [2]:
import pandas as pd

---

# Pará

In [3]:
df_pa = pd.read_csv('./dados-tratados/20240205145613-dados_VCM_PA.csv', sep='|')

In [4]:
df_pa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7856 entries, 0 to 7855
Data columns (total 33 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   data_registro           7856 non-null   object 
 1   hora_registro           7856 non-null   object 
 2   data_fato               7856 non-null   object 
 3   hora_fato               7856 non-null   object 
 4   dia_semana              7856 non-null   object 
 5   faixa_horaria_2h        7856 non-null   object 
 6   faixa_horaria_6h        7856 non-null   object 
 7   classe_motivo           7853 non-null   object 
 8   mes_fato                7856 non-null   int64  
 9   ano_registro            7856 non-null   int64  
 10  ano_fato                7856 non-null   int64  
 11  registros               7856 non-null   object 
 12  municipio               7856 non-null   object 
 13  bairro                  7856 non-null   object 
 14  local_ocorrencia        7856 non-null   

In [5]:
df_pa.head()

,data_registro,hora_registro,data_fato,hora_fato,dia_semana,faixa_horaria_2h,faixa_horaria_6h,classe_motivo,mes_fato,ano_registro,...,vit_profissao,aut_dt_nasc,aut_idade,aut_faixa_etaria,aut_cor_pele,aut_grau_inst,aut_profissao,grau_de_relacionamento,faixa_horaria,estado
0,2020-02-10,13:26:00,2020-02-09,13:26:00,DOM,12h - 14h,12h - 18h,CRIMES DE VIOLENCIA DOMESTICA - LEI 11.340/2006,2,2020,...,NAO INFORMADO,NaN,NaN,Desconhecida,NaN,NaN,NaN,NaN,Tarde,PA
1,2020-01-10,16:34:00,2020-01-06,16:00:00,SEG,16h - 18h,12h - 18h,CRIMES DE VIOLENCIA DOMESTICA - LEI 11.340/2006,1,2020,...,NAO INFORMADO,NaN,NaN,Desconhecida,NaN,NaN,NaN,NaN,Tarde,PA
2,2020-07-02,09:56:00,2020-07-02,09:56:00,QUI,08h - 10h,06h - 12h,CRIMES DE VIOLENCIA DOMESTICA - LEI 11.340/2006,7,2020,...,DOMESTICA,NaN,NaN,Desconhecida,NaN,NaN,NaN,NaN,Manhã,PA
3,2022-04-07,16:47:00,2022-04-07,16:47:00,QUI,16h - 18h,12h - 18h,CRIMES DE VIOLENCIA DOMESTICA - LEI 11.340/2006,4,2022,...,NaN,NaN,NaN,Desconhecida,PREJUDICADO,NaN,NaN,NaN,Tarde,PA
4,2022-03-16,18:27:00,2022-03-16,18:27:00,QUA,18h - 20h,18h - 00h,CRIMES DE VIOLENCIA DOMESTICA - LEI 11.340/2006,3,2022,...,PREJUDICADO,NaN,NaN,Desconhecida,PREJUDICADO,PREJUDICADO,PREJUDICADO,EX-COMPANHEIRO(A),Noite,PA


In [6]:
df_pa.drop(columns=['especificacao_crime'], inplace=True)

In [7]:
print(df_pa.aut_idade.isnull().sum())

print(len(df_pa.aut_idade))

7761
7856


In [8]:
print(df_pa.columns)

Index(['data_registro', 'hora_registro', 'data_fato', 'hora_fato',
       'dia_semana', 'faixa_horaria_2h', 'faixa_horaria_6h', 'classe_motivo',
       'mes_fato', 'ano_registro', 'ano_fato', 'registros', 'municipio',
       'bairro', 'local_ocorrencia', 'meio_empregado', 'mes_registro',
       'vit_dt_nasc', 'vit_idade', 'vit_faixa_etaria', 'vit_cor_pele',
       'vit_grau_inst', 'vit_profissao', 'aut_dt_nasc', 'aut_idade',
       'aut_faixa_etaria', 'aut_cor_pele', 'aut_grau_inst', 'aut_profissao',
       'grau_de_relacionamento', 'faixa_horaria', 'estado'],
      dtype='object')


In [9]:
# Contagem do número de variáveis
v = len(df_pa.columns)
# Contagem do número de registros
n = len(df_pa)

print(f'O conjunto de dados do Pará possui {n} registros e {v} variáveis.')

O conjunto de dados do Pará possui 7856 registros e 32 variáveis.


----

# Paraná

In [15]:
df_pr = pd.read_csv('./dados-tratados/20240208095427-df_VCM_PR-pandas.csv', sep='|')

In [16]:
df_pr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17698 entries, 0 to 17697
Data columns (total 35 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   data_registro           17698 non-null  object 
 1   hora_registro           17698 non-null  object 
 2   data_fato               17698 non-null  object 
 3   hora_fato               17698 non-null  object 
 4   dia_semana              17698 non-null  object 
 5   faixa_horaria_2h        17698 non-null  object 
 6   faixa_horaria_6h        17698 non-null  object 
 7   classe_motivo           17698 non-null  object 
 8   mes_fato                17698 non-null  int64  
 9   ano_registro            17698 non-null  int64  
 10  ano_fato                17698 non-null  int64  
 11  registros               17698 non-null  object 
 12  municipio               17697 non-null  object 
 13  bairro                  17697 non-null  object 
 14  local_ocorrencia        17698 non-null

In [17]:
print(df_pr.columns)

Index(['data_registro', 'hora_registro', 'data_fato', 'hora_fato',
       'dia_semana', 'faixa_horaria_2h', 'faixa_horaria_6h', 'classe_motivo',
       'mes_fato', 'ano_registro', 'ano_fato', 'registros', 'municipio',
       'bairro', 'local_ocorrencia', 'meio_empregado', 'mes_registro',
       'vit_dt_nasc', 'vit_idade', 'vit_faixa_etaria', 'vit_cor_pele',
       'vit_grau_inst', 'vit_profissao', 'aut_dt_nasc', 'aut_idade',
       'aut_faixa_etaria', 'aut_cor_pele', 'aut_grau_inst', 'aut_profissao',
       'grau_de_relacionamento', 'faixa_horaria', 'estado', 'id_boletim',
       'vitima', 'autor'],
      dtype='object')


In [18]:
# Contagem do número de variáveis
v = len(df_pr.columns)
# Contagem do número de registros
n = len(df_pr)

print(f'O conjunto de dados do Paraná possui {n} registros e {v} variáveis.')

O conjunto de dados do Paraná possui 17698 registros e 35 variáveis.


---

# Comparação entre os dois conjuntos

Antes da comparação, é necessário adicionar as três colunas presentes nos dados do Paraná que não estão nos dados do Pará.

In [19]:
df_pa['id_boletim'] = None
df_pa['vitima'] = None
df_pa['autor'] = None

In [20]:
# Verificando se são as mesmas colunas
print(df_pa.columns == df_pr.columns)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True]


Verificando o total da base, com a fusão dos dois conjuntos.

In [21]:
len(df_pa) + len(df_pr)

25554